In [2]:
import os
import pandas as pd
import re
from tqdm import tqdm

def build_basic_data(raw_path='./csv/'):
    raw_files = os.listdir(raw_path)
    if not raw_path:
        return
    data = []
    for raw_file_name in tqdm(raw_files):
        raw_file = pd.read_csv(os.path.join(raw_path, raw_file_name), header = None)
        parking_name = raw_file_name[:-4]
        
        loc_str = raw_file[1][2]
        start_co = loc_str.rindex('：')
        longitude, latitude = loc_str[start_co+1:].split(',')
        total_space = int(raw_file[1][3])
        monthly_fee = try_get_monthly_fee(raw_file[1][5])
        
        building_type = raw_file[1][6]
        data.append([parking_name, float(latitude), float(longitude), int(total_space), int(monthly_fee), building_type, (float(latitude), float(longitude))])
    df = pd.DataFrame(data, columns=['parking_name', 'latitude', 'longitude', 'total_space','monthly_fee','building_type', 'lat_long'])
    return df
    

def try_get_monthly_fee(fee_str):
    monthly_fee = 300  # 300 is monthly mean
    fee_str = str(fee_str)
    if fee_str:
        fee_str = re.findall('\d+', fee_str)
    if fee_str:
        monthly_fee = int(fee_str[0])
    if monthly_fee < 50: # daily fee convert to monthly fee, 300 is monthly mean, 10 is hourly mean
        monthly_fee = 300*monthly_fee/10
    return monthly_fee


def xlsx2csv(xlsx_path='./xlsx'):
    raw_files = os.listdir(xlsx_path)
    if not xlsx_path:
        return
    save_path = './csv/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    data = []
    print(raw_files)
    for raw_file in tqdm(raw_files):
        new_file = save_path+os.path.splitext(raw_file)[0]+'.csv'
        if os.path.exists(new_file):
            continue
        print(raw_file)
        raw_excel = pd.read_excel(os.path.join(xlsx_path, raw_file), header = None).drop(0, 1)
        raw_excel.to_csv(new_file,encoding='utf_8_sig', index=False, header=False)
    print("finished xlsx2csv!")

In [3]:
# basic_info_df = build_basic_data()
basic_info_df = pd.read_csv('parkings_info.csv')
basic_info_df['lat_long'] = list(zip(basic_info_df['latitude'], basic_info_df['longitude']))

In [31]:
basic_info_df.to_csv('parkings_info.csv',encoding='utf_8_sig', index=False)

In [4]:
import folium

from folium import IFrame

def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

# Instantiate a feature group for the parkings in the dataframe
parkings = folium.map.FeatureGroup()
target = basic_info_df.iloc[0]
# Loop through and add each to the parkings feature group
for lat, lng, name in zip(basic_info_df.latitude, basic_info_df.longitude, basic_info_df.parking_name):
    parkings.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color= ('red' if name==target.parking_name else 'green'),
            fill_opacity=0.6,
            popup=parse_zhch(name),
            encode='uft-8'))

luohu_map = folium.Map(location=[target.latitude, target.longitude], zoom_start=13)
luohu_map.add_child(parkings)

In [6]:
left_top = basic_info_df.loc[basic_info_df["parking_name"] =='宝丽大厦'].iloc[0]
right_bottom = basic_info_df.loc[basic_info_df["parking_name"] =='合作金融大厦'].iloc[0]
basic_info_df[(basic_info_df.parking_name == '宝丽大厦') | (basic_info_df.parking_name == '合作金融大厦')]

,parking_name,latitude,longitude,total_space,monthly_fee,building_type,lat_long
29,合作金融大厦,22.549830,114.123911,26,900,商业,"(22.54983, 114.123911)"
39,宝丽大厦,22.553417,114.116950,73,110,住宅,"(22.553417, 114.11695)"


In [7]:
from geopy import distance
# build area for points near than max_dist to target
def get_area(target, max_dis, all_df):
    area_data = []
    for index, row in all_df.iterrows():
        if distance.distance(row.lat_long,target.lat_long).km<max_dis:
            area_data.append(row.values.tolist())

    return pd.DataFrame(area_data, columns=all_df.columns.values.tolist())

In [9]:
# from geopy import distance

# total = 0
# total_dis = 0.0
# for lat1, long1, name1 in zip(area1.latitude, area1.longitude, area1.parking_name):
#     for lat2, long2, name2 in zip(area1.latitude, area1.longitude, area1.parking_name):
#         if name1 == name2:
#             continue
#         total+=1
#         total_dis+=distance.distance((lat1, long1), (lat2, long2)).km

# avg_dis = total_dis/total
# print(total_dis, avg_dis)

In [8]:
# build connection bettween points near than max_dist
def build_adj_matrix(area, max_dist=0.4):
    adj_matrix = []
    name_key_dict = dict()
    key_name_dict = dict()
    key = 0
    for lat1, long1, name1 in zip(area.latitude, area.longitude, area.parking_name):
        name_key_dict[name1]=key
        key_name_dict[key]=name1
        key+=1
        one_line=[]
        for lat2, long2, name2 in zip(area.latitude, area.longitude, area.parking_name):
            if name1 != name2 and distance.distance((lat1, long1), (lat2, long2)).km<max_dist:
                one_line.append(1)
            else:
                one_line.append(0)
        adj_matrix.append(one_line)
    return adj_matrix, name_key_dict, key_name_dict

In [9]:
import folium

def build_adj_map(target, area_, adj_mat, key_names, name_keys):
    # Instantiate a feature group for the parkings in the dataframe
    parkings = folium.map.FeatureGroup()

    # Loop through and add each to the parkings feature group
    for lat, lng, name in zip(area_.latitude, area_.longitude, area_.parking_name):
        parkings.add_child(
            folium.CircleMarker(
                [lat, lng],
                radius=5, # define how big you want the circle markers to be
                color='yellow',
                fill=True,
                fill_color= ('red' if name==target.parking_name else 'green'),
                fill_opacity=0.6,
                popup=parse_zhch(name),
                encode='uft-8'))
        neighbor_index = 0
        for is_neighbor in adj_mat[name_keys[name]]:
            if is_neighbor == 1:
                start = (lat, lng)
                end_parking = area_.loc[area_["parking_name"] == key_names[neighbor_index]].iloc[0]
                end = (end_parking.latitude, end_parking.longitude)
                parkings.add_child(folium.PolyLine([start, end], color="blue", weight=1.5, opacity=0.2))
            neighbor_index+=1
    
    luohu_map = folium.Map(location=[target.latitude, target.longitude], zoom_start=16)
    luohu_map.add_child(parkings)
    return luohu_map

In [10]:
def build_graph(parking_name, max_dis=0.5, conn_coe=3/4, zoom_start=16):
    target_parking = basic_info_df.loc[basic_info_df.parking_name == parking_name].iloc[0]
    area = get_area(target_parking, max_dis, basic_info_df)
    adj_mat, nks, kns = build_adj_matrix(area, max_dis*conn_coe)
    target_map = build_adj_map(target_parking, area, adj_mat, kns, nks)
    return area, adj_mat, target_map, nks, kns

In [11]:
target_area, adj_matrix, target_map, nks, kns = build_graph('缤纷时代家园')

In [19]:
target_map

In [ ]:
# 以下是预处理成seq

In [20]:
import pandas as pd
import datetime
import os
from numpy import nan as NaN

def parse_date(v):
    try:
        return datetime.datetime.strptime(v.replace('/','-')[:19], '%Y-%m-%d %H:%M:%S')
    except Exception as ex:
        # print(str(ex))
        pass
    return NaN

def get_in_out(filename):
    if not filename:
        return
    
    # dateparser = lambda x: datetime.datetime.strptime(x.replace('/','-')[:19], '%Y-%m-%d %H:%M:%S')
    seq_df_raw = pd.read_csv(filename, parse_dates=[0,1], date_parser=parse_date, skiprows=8)
    seq_df = seq_df_raw.dropna()
    if len(seq_df) != len(seq_df_raw):
        print('droping rows', len(seq_df_raw)-len(seq_df))
    seq_df.columns = ['in_', 'out']
    seq_df = seq_df.sort_values(by=['in_'], ascending=True)
    seq_df['in_'].values.sort()
    ins = [(x, 1) for x in seq_df['in_'].values]

    seq_df['out'].values.sort()
    outs = [(x, -1) for x in seq_df['out'].values]
    return ins, outs

In [21]:
def count_between(start_index, end_date, ary):
    count = 0
    scope = ary[start_index:-1]
    for d in scope:
        if d[0] >= ary[start_index][0] and d[0] < end_date:
            count+=1
        else:
            break
    return count    

In [22]:
def find_index_just_later(ary, date, current_index=0):
    if len(ary)<=1:
        return current_index+1
    if ary[0][0] <= date and ary[1][0] >= date:
        return current_index+1
    
    half = int(len(ary)/2)
    if date < ary[half][0]:
        return find_index_just_later(ary[0:half], date, current_index)
    else:
        return find_index_just_later(ary[half:], date, current_index+half)

In [23]:
import numpy as np
import pandas as pd

def build_seq(ins, outs):
    start_date = np.datetime64('2016-01-01')
    end_date = outs[-1][0]
    occupy_seq = []
    current_date = np.datetime64(pd.to_datetime(start_date).date()) # cut off time,start with 2016-01-01 00:00:00 date only
    current_in_index = find_index_just_later(ins, start_date)
    current_out_index = find_index_just_later(outs, start_date)
    current_occupy = current_in_index-current_out_index
    while current_date < end_date:
        in_count = count_between(current_in_index, current_date, ins)
        if in_count > 0:
            current_occupy+=in_count
            current_in_index+=in_count
        out_count = count_between(current_out_index, current_date, outs)
        if out_count > 0:
            current_occupy-=out_count
            current_out_index+=out_count
        occupy_seq.append([current_date,current_occupy])
        current_date+=np.timedelta64(15,'m')
    return pd.DataFrame(occupy_seq, columns=['date','occupy'])

In [24]:
import numpy as np
from tqdm import tqdm
import os
from matplotlib import pyplot as plt

def build_seq_data(raw_path='./csv/'):
    raw_files = os.listdir(raw_path)
    if not raw_path:
        return
    for raw_file_name in tqdm(raw_files):
        parking_name = raw_file_name[:-4]
        img_file = 'images/'+parking_name+'.png'
        if os.path.isfile(img_file):
            print('skipping', raw_file_name)
            continue
        print('building', raw_file_name)
        ins, outs = get_in_out(os.path.join(raw_path, raw_file_name))
        occupy_seq = build_seq(ins, outs)
        if len(occupy_seq) == 0:
            print('no valid data for', raw_file_name)
            continue
        occupy_seq.to_csv('seqs/'+parking_name+'_seq.csv',encoding='utf_8_sig', index=False)
        fig = occupy_seq.plot(x='date', figsize=(100,10), fontsize=40).get_figure()
        fig.savefig(img_file)
        plt.clf()
        

In [183]:
build_seq_data()

  0%|                                                                 | 0/100 [00:00<?, ?it/s]

building 万山珠宝工业园.csv


  1%|▌                                                        | 1/100 [00:33<54:59, 33.33s/it]

building 万达丰大厦.csv


  2%|█                                                      | 2/100 [01:58<1:20:04, 49.03s/it]

...

<Figure size 7200x720 with 0 Axes>

<Figure size 7200x720 with 0 Axes>

<Figure size 7200x720 with 0 Axes>

<Figure size 7200x720 with 0 Axes>

<Figure size 7200x720 with 0 Axes>

In [25]:
target_area

,parking_name,latitude,longitude,total_space,monthly_fee,building_type,lat_long
0,万达丰大厦,22.558109,114.131096,97,250,写字楼,"(22.558108999999998, 114.131096)"
1,东悦名轩,22.557403,114.129421,168,250,住宅,"(22.557403, 114.129421)"
2,东方华都,22.557188,114.135247,191,250,写字楼,"(22.557188, 114.13524699999999)"
3,东方大厦,22.557891,114.130766,37,480,写字楼,"(22.557891, 114.130766)"
4,东方颐园,22.555972,114.133380,131,250,住宅,"(22.555972, 114.13338)"
5,东门天地大厦,22.555801,114.128255,241,250,写字楼,"(22.555801000000002, 114.12825500000001)"
6,丰园酒店,22.552875,114.130616,140,350,住宅,"(22.552875, 114.13061599999999)"
7,京基晶都酒店,22.555972,114.133380,155,500,商业,"(22.555972, 114.13338)"
8,华隆园,22.557172,114.133254,70,250,写字楼,"(22.557172, 114.13325400000001)"
9,名都大厦,22.556247,114.136144,40,250,写字楼,"(22.556247, 114.13614399999999)"


In [12]:
from IPython.display import Image, display
from datetime import date, timedelta
import pandas as pd

# 整合到一个文件中
area_df = pd.DataFrame()
for name in target_area.parking_name:
    file_name = 'archive/seqs/'+name+'_seq.csv'
    file_df = pd.read_csv(file_name)
    file_df['parking'] = nks[name]
    cols = file_df.columns.tolist()
    cols = [cols[0], cols[2], cols[1]]
    file_df = file_df[cols]
    if len(area_df)>0:
        area_df = pd.concat([area_df, file_df])
    else:
        area_df = file_df

In [ ]:
import matplotlib.pyplot as plt

plt.plot_date(data=area_df, y = 'occupy', x='date')

In [ ]:
# todo: try build network input schema
# 目标：预测电影大厦停车场11月份的数据

In [2]:
import numpy as np

start = np.datetime64('2016-08-01')
end = np.datetime64('2017-01-01')

In [ ]:

area_df

In [17]:
from shutil import copyfile

to_model = '../GCN-GRU-MC-GAN/generated/data1/'
for name in target_area.parking_name:
    file_name = 'archive/seqs/'+name+'_seq.csv'
    copyfile(file_name, to_model+'area_seqs/'+name+'_seq.csv')
copyfile('parkings_info.csv', to_model+'parkings_info.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../GCN-GRU-MC-GAN/generated/data1/area_seqs/万达丰大厦_seq.csv'